# Import necessary packages

In [172]:
import copy
import sympy as sp
from sympy.matrices import ones, eye
import re
import dimod
import neal
from dwave.system import DWaveSampler, EmbeddingComposite

# Input Variables

coords_original - Coordinates considered for distance measure in the hubo<br>torsional_bonds - Torsional bond numbers and their respective coordinate ends<br>coords_rotation_dict - Coordinate numbers and the bonds that affect them<br>distance_pairs - Pairs of coordinates to find distance

In [173]:
coords_original = {
    0: [1.0, -0.5, 0.0],
    1: [0.0, 0.0, 0.0],
    2: [0.0, 1.0, 0.0],
    3: [1.0, 1.5, 0.0],
    4: [2.0, 1.0, 0.0]
}
torsional_bonds = {
    0: (1, 2),
    1: (2, 3)
}
coords_rotation_dict = {
    0: [],
    1: [],
    2: [],
    3: [0],
    4: [0, 1]
}
distance_pairs = [
    (0, 3),
    (0, 4),
    (1, 3),
    (1, 4),
    (2, 4)]

In [174]:
coords_dict = copy.deepcopy(coords_original)
final_coords = copy.deepcopy(coords_original)

n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles

x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [175]:
def generate_hard_constraint():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

In [176]:
hard_hubo = generate_hard_constraint()
sp.pprint(hard_hubo)

        ⎛                                           2                         
A_const⋅⎝(x₀ + x₁ + x₂ + x₃ + x₄ + x₅ + x₆ + x₇ - 1)  + (x₁₀ + x₁₁ + x₁₂ + x₁₃

                           2⎞
 + x₁₄ + x₁₅ + x₈ + x₉ - 1) ⎠


# Functions to rotate coordinates with hubo variables

In [177]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [178]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [179]:
def generate_rotation_matrix(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [180]:
def rotate_all_coordinates():
    for i in coords_original.keys():
        if len(coords_rotation_dict[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict[i]:
                temp_rot_mat = generate_rotation_matrix(coords_original[torsional_bonds[bond_no][0]],
                                                        coords_original[torsional_bonds[bond_no][1]], bond_no)
                rot_mat = rot_mat * temp_rot_mat
            coords_dict[i] = rotate_coordinates(rot_mat, coords_original[i])

In [181]:
rotate_all_coordinates()

In [182]:
sp.pprint(coords_original)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0, 1.5, 0.
0], 4: [2.0, 1.0, 0.0]}


In [183]:
sp.pprint(coords_dict)

{0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0⋅x₀ + 0.5
⋅√2⋅x₁ - 0.5⋅√2⋅x₃ - x₄ - 0.5⋅√2⋅x₅ + 0.5⋅√2⋅x₇, 1.5, -0.5⋅√2⋅x₁ - x₂ - 0.5⋅√2
⋅x₃ + 0.5⋅√2⋅x₅ + 1.0⋅x₆ + 0.5⋅√2⋅x₇], 4: [-0.2⋅√2⋅x₀⋅x₁₁ - 0.4⋅x₀⋅x₁₂ - 0.2⋅√
2⋅x₀⋅x₁₃ + 0.2⋅√2⋅x₀⋅x₁₅ + 0.4⋅x₀⋅x₈ + 0.2⋅√2⋅x₀⋅x₉ + 1.6⋅x₀ - 0.4472135954999
58⋅√2⋅x₁⋅x₁₀ - 0.647213595499958⋅x₁⋅x₁₁ - 0.2⋅√2⋅x₁⋅x₁₂ + 0.247213595499958⋅x₁
⋅x₁₃ + 0.447213595499958⋅√2⋅x₁⋅x₁₄ + 0.647213595499958⋅x₁⋅x₁₅ + 0.2⋅√2⋅x₁⋅x₈ -
 0.247213595499958⋅x₁⋅x₉ + 0.8⋅√2⋅x₁ - 0.894427190999916⋅x₁₀⋅x₂ - 0.4472135954
99958⋅√2⋅x₁₀⋅x₃ + 0.447213595499958⋅√2⋅x₁₀⋅x₅ + 0.894427190999916⋅x₁₀⋅x₆ + 0.4
47213595499958⋅√2⋅x₁₀⋅x₇ - 0.447213595499958⋅√2⋅x₁₁⋅x₂ - 0.247213595499958⋅x₁₁
⋅x₃ + 0.2⋅√2⋅x₁₁⋅x₄ + 0.647213595499958⋅x₁₁⋅x₅ + 0.447213595499958⋅√2⋅x₁₁⋅x₆ +
 0.247213595499958⋅x₁₁⋅x₇ + 0.2⋅√2⋅x₁₂⋅x₃ + 0.4⋅x₁₂⋅x₄ + 0.2⋅√2⋅x₁₂⋅x₅ - 0.2⋅√
2⋅x₁₂⋅x₇ + 0.447213595499958⋅√2⋅x₁₃⋅x₂ + 0.647213595499958⋅x₁₃⋅x₃ + 0.2⋅√2⋅x₁₃
⋅x₄ - 0.247213595499958⋅x₁₃⋅x₅ - 0.447213595499958⋅√

# Functions to generate the optimization contraint in hubo

In [184]:
def distance_squared(first_coords, second_coords):
    dis_sq = sp.expand((second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 +(second_coords[2] - first_coords[2])**2)
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_dict[pair[0]], coords_dict[pair[1]])
    return distance_sq.expand()

In [185]:
distance_hubo = generate_distance_hubo()
sp.pprint(distance_hubo)

⋅x₉ - 3.84⋅x₁₁⋅x₄⋅x₅ + 0.96⋅x₁₁⋅x₄⋅x₇⋅x₈ - 1.92⋅√2⋅x₁₁⋅x₄⋅x₇⋅

                                                    2                 2       
x₉ + 3.84⋅x₁₁⋅x₄⋅x₇ - 0.4⋅√2⋅x₁₁⋅x₄ - 0.48⋅√2⋅x₁₁⋅x₅ ⋅x₈ + 1.92⋅x₁₁⋅x₅ ⋅x₉ - 1

             2                                                                
.92⋅√2⋅x₁₁⋅x₅  - 0.96⋅x₁₁⋅x₅⋅x₆⋅x₈ + 1.92⋅√2⋅x₁₁⋅x₅⋅x₆⋅x₉ - 3.84⋅x₁₁⋅x₅⋅x₆ - 1

                                       2                 2                    
.29442719099992⋅x₁₁⋅x₅ - 0.48⋅√2⋅x₁₁⋅x₆ ⋅x₈ + 1.92⋅x₁₁⋅x₆ ⋅x₉ - 1.92⋅√2⋅x₁₁⋅x₆

2                                                                             
  - 0.96⋅x₁₁⋅x₆⋅x₇⋅x₈ + 1.92⋅√2⋅x₁₁⋅x₆⋅x₇⋅x₉ - 3.84⋅x₁₁⋅x₆⋅x₇ - 0.894427190999

                              2                 2                    2        
916⋅√2⋅x₁₁⋅x₆ - 0.48⋅√2⋅x₁₁⋅x₇ ⋅x₈ + 1.92⋅x₁₁⋅x₇ ⋅x₉ - 1.92⋅√2⋅x₁₁⋅x₇  - 0.494

                                                                           2  
427190999916⋅x₁₁⋅x₇ - 1.92⋅√2⋅x₁₁⋅x₈ - 1.92⋅x₁₁⋅x₉ + 3.92⋅√2⋅x₁

# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [186]:
hubo_expr = sp.expand(hard_hubo - distance_hubo)
hubo_expr

x2 - 0.96*x1*x11*x4*x8 + 1.92*sqrt(2)*x1*x11*x4*x9 - 3.84*x1*x11*x4 - 0.960000000000001*sqrt(2)*x1*x11*x5*x8 + 3.84*x1*x11*x5*x9 - 3.84*sqrt(2)*x1*x11*x5 - 0.96*x1*x11*x6*x8 + 1.92*sqrt(2)*x1*x11*x6*x9 - 3.84*x1*x11*x6 - 1.29442719099992*x1*x11 - 0.48*sqrt(2)*x1*x12**2*x2 + 0.48*sqrt(2)*x1*x12**2*x4 + 0.96*x1*x12**2*x5 + 0.48*sqrt(2)*x1*x12**2*x6 - 0.96*x1*x12*x13*x2 + 0.96*x1*x12*x13*x4 + 0.960000000000001*sqrt(2)*x1*x12*x13*x5 + 0.96*x1*x12*x13*x6 + 0.96*x1*x12*x15*x2 - 0.96*x1*x12*x15*x4 - 0.960000000000001*sqrt(2)*x1*x12*x15*x5 - 0.96*x1*x12*x15*x6 + 0.96*sqrt(2)*x1*x12*x2*x8 + 0.96*x1*x12*x2*x9 + 3.84*sqrt(2)*x1*x12*x2 - 0.96*sqrt(2)*x1*x12*x4*x8 - 0.96*x1*x12*x4*x9 - 3.84*sqrt(2)*x1*x12*x4 - 1.92*x1*x12*x5*x8 - 0.960000000000001*sqrt(2)*x1*x12*x5*x9 - 7.68*x1*x12*x5 - 0.96*sqrt(2)*x1*x12*x6*x8 - 0.96*x1*x12*x6*x9 - 3.84*sqrt(2)*x1*x12*x6 - 0.4*sqrt(2)*x1*x12 - 1.44*sqrt(2)*x1*x13**2*x2 + 1.44*sqrt(2)*x1*x13**2*x4 + 2.88*x1*x13**2*x5 + 1.44*sqrt(2)*x1*x13**2*x6 - 4.8*x1*x13*x14*x2

In [217]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [218]:
hubo_dict = hubo_expr_to_dict()
print(hubo_dict)

): 4.80000000000000, (2, 3, 10, 13): 4.80000000000000, (3, 4, 10, 13): 4.80000000000000, (4, 5, 10, 13): 4.80000000000000, (5, 6, 10, 13): 4.80000000000000, (6, 7, 10, 13): 4.80000000000000, (2, 3, 10, 15): 4.80000000000000, (3, 4, 10, 15): 4.80000000000000, (4, 5, 10, 15): 4.80000000000000, (5, 6, 10, 15): 4.80000000000000, (6, 7, 10, 15): 4.80000000000000, (2, 5, 9, 10): 4.80000000000000, (2, 7, 9, 10): 4.80000000000000, (3, 6, 9, 10): 4.80000000000000, (4, 7, 9, 10): 4.80000000000000, (2, 3, 11, 14): 4.80000000000000, (3, 4, 11, 14): 4.80000000000000, (4, 5, 11, 14): 4.80000000000000, (5, 6, 11, 14): 4.80000000000000, (6, 7, 11, 14): 4.80000000000000, (2, 5, 13, 14): 4.80000000000000, (2, 7, 13, 14): 4.80000000000000, (3, 6, 13, 14): 4.80000000000000, (4, 7, 13, 14): 4.80000000000000, (2, 5, 14, 15): 4.80000000000000, (2, 7, 14, 15): 4.80000000000000, (3, 6, 14, 15): 4.80000000000000, (4, 7, 14, 15): 4.80000000000000, (2, 3, 9, 14): 4.80000000000000, (3, 4, 9, 14): 4.80000000000000,

In [219]:
file_name = "hubo_dict.txt"
f = open(file_name, "w")
print('full hubo dictionary written in file - hubo_dict.txt')
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - hubo_dict.txt


In [220]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(file_name, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(file_name, 'r').read()
HUBO=eval(read_dictionary)

In [221]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 969


In [222]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [223]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [224]:
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [225]:
sampler = neal.SimulatedAnnealingSampler()

sample_size=10

In [226]:
sampleset = sampler.sample(bqm, num_reads=sample_size)
print(sampleset)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15     energy num_oc.
3  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0 -37.280672       1
9  0  0  0  0  0  1  0  0  0  0  0  0  0  1  0  0 -37.280672       1
4  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0     -36.86       1
1  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0 -33.603717       1
0  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -29.926762       1
7  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -29.926762       1
2  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1 -26.193238       1
8  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0 -24.856955       1
5  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  1 -22.516283       1
6  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0     -21.18       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [227]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=1000)
print(sampleset.first.sample)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
